In [1]:
import h5py
import numpy as np
import operator
import visualisation
import pandas as pd
import matplotlib.pyplot as plt
from dtw import dtw

from sporco import plot, util
from sporco.admm import cbpdn
from sporco.dictlrn import cbpdndl
from numpy.fft import rfft, rfftfreq
from itertools import tee

import scipy.signal as sg
import pandas as pd
from scipy.signal import butter, lfilter, freqz



plt.rcParams["figure.figsize"] = (16,8)


## Load the data

In [2]:
PATH_TO_TESTING_DATA = "additional_files_dreem/X_train_denoised_array.h5"
PATH_TO_TRAINING_TARGET = "data/y_train_tX9Br0C.csv"
h5_file = h5py.File(PATH_TO_TESTING_DATA)
mask = np.array(pd.read_csv(PATH_TO_TRAINING_TARGET))

/Users/Claudia/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
X_train = np.array(h5_file.get('data'))

In [4]:
X_train.shape

(8, 4400, 9000)

Separation of signals containing at least one apnea event and those who do do not.

In [5]:
idx_apnea_signals = []
idx_normal_signals = []
for i in range(4400):
    if 1 in mask[i,1:]:
        idx_apnea_signals += [i]
    else:
        idx_normal_signals += [i]

## Exploration of respiratory signals for a normal sleep

In [ ]:
n_signals = 5
chosen_signals = np.random.randint(len(idx_normal_signals),size=n_signals)

fig, ax = plt.subplots(nrows=n_signals, figsize=(20,10))
for i in range(n_signals):
    ax[i].plot(X_train[0,idx_normal_signals[chosen_signals[i]]])
    ax[i].set_title(f"Signal number {chosen_signals[i]}")

In [ ]:
fig, ax = plt.subplots(nrows=n_signals, figsize=(20,10))
for i in range(n_signals):
    ax[i].plot(X_train[1,idx_normal_signals[chosen_signals[i]]])
    ax[i].set_title(f"Signal number {chosen_signals[i]}")

In [ ]:
fig, ax = plt.subplots(nrows=n_signals, figsize=(20,10))
for i in range(n_signals):
    ax[i].plot(X_train[2,idx_normal_signals[chosen_signals[i]]])
    ax[i].set_title(f"Signal number {chosen_signals[i]}")

In [ ]:
fig, ax = plt.subplots(nrows=n_signals, figsize=(20,10))
for i in range(n_signals):
    ax[i].plot(X_train[3,idx_normal_signals[chosen_signals[i]]])
    ax[i].set_title(f"Signal number {chosen_signals[i]}")

## Utility functions for CDL

In [ ]:
def plot_CDL(signal, Z, D, figsize=(15, 10)):
    """Plot the learned dictionary `D` and the associated sparse codes `Z`.

    `signal` is an univariate signal of shape (n_samples,) or (n_samples, 1).
    """
    (atom_length, n_atoms) = np.shape(D)
    plt.figure(figsize=figsize)
    plt.subplot(n_atoms + 1, 3, (2, 3))
    plt.plot(signal)
    for i in range(n_atoms):
        plt.subplot(n_atoms + 1, 3, 3 * i + 4)
        plt.plot(D[:, i])
        plt.subplot(n_atoms + 1, 3, (3 * i + 5, 3 * i + 6))
        plt.plot(Z[:, i])
        plt.ylim((np.min(Z), np.max(Z)))

In [ ]:
def atleast_2d(ary):
    """Reshape array to at least two dimensions."""
    if ary.ndim == 0:
        return ary.reshape(1, 1)
    elif ary.ndim == 1:
        return ary[:, np.newaxis]
    return ary

In [ ]:
def display_distance_matrix_as_table(
    distance_matrix, labels=None, figsize=(8, 2)
):
    fig, ax = plt.subplots(figsize=figsize)
    ax.axis("tight")
    ax.axis("off")
    norm = mpl.colors.Normalize()
    cell_colours_hex = np.empty(shape=distance_matrix.shape, dtype=object)
    cell_colours_rgba = plt.get_cmap("magma")(norm(distance_matrix))

    for i in range(distance_matrix.shape[0]):
        for j in range(i + 1, distance_matrix.shape[0]):
            cell_colours_hex[i, j] = rgb2hex(
                cell_colours_rgba[i, j], keep_alpha=True
            )
            cell_colours_hex[j, i] = cell_colours_hex[i, j]

    if labels is not None:
        _ = ax.table(
            cellText=distance_matrix,
            colLabels=labels,
            rowLabels=labels,
            loc="center",
            cellColours=cell_colours_hex,
        )
    else:
        _ = ax.table(
            cellText=distance_matrix,
            loc="center",
            cellColours=cell_colours_hex,
        )

    return ax

def fig_ax(figsize=(15, 5)):
    return plt.subplots(figsize=figsize)

## CDL on a single signal

For a 1D signal $\mathbf{x}\in\mathbb{R}^N$ with $N$ samples, the convolutional dictionary learning tasks amounts to solving the following optimization problem:

$$
\min_{(\mathbf{d}_k)_k, (\mathbf{z}_k)_k \\ \lVert\mathbf{d}_k\rVert^2\leq 1} \quad\left\lVert \mathbf{x} - \sum_{k=1}^K \mathbf{z}_k * \mathbf{d}_k \right\rVert^2 \quad + \quad\lambda \sum_{k=1}^K \lVert\mathbf{z}_k\rVert_1
$$

where $\mathbf{d}_k\in\mathbb{R}^L$ are the $K$ dictionary atoms (patterns), $\mathbf{z}_k\in\mathbb{R}^{N-L+1}$ are activations signals, and $\lambda>0$ is the sparsity constraint.

This problem is not convex with respect to the couple $(\mathbf{d}_k)_k, (\mathbf{z}_k)_k$ but convex when the subproblems are taken individually.

In [ ]:
# In this cell, we set parameters and options that should probably remained unchanged
PENALTY = 3

# options for the dictionary learning and sparse coding procedures
def get_opt_dl(penalty=PENALTY):
    """Return the option class for the dictionary learning"""
    return cbpdndl.ConvBPDNDictLearn.Options(
        {
            "Verbose": False,
            "MaxMainIter": 50,
            "CBPDN": {"rho": 50.0 * penalty + 0.5, "NonNegCoef": True},
            "CCMOD": {"rho": 10.0},
        },
        dmethod="cns",
    )


def get_opt_sc():
    """Return the option class for the sparse coding"""
    return cbpdn.ConvBPDN.Options(
        {
            "Verbose": False,
            "MaxMainIter": 50,
            "RelStopTol": 5e-3,
            "AuxVarObj": False,
            "NonNegCoef": True,  # only positive sparse codes
        }
    )

Let's apply dictionary learning on the first normal signal.

In [ ]:
labels = ["Abdominal belt", "Airflow", "PPG", "Thoracic belt", "Snoring indicator", "SPO2", "C4-A1", "O2-A1"]

fig, ax = plt.subplots(nrows=4, figsize=(20,8))
for i in range(4):
    ax[i].plot(X_train[i, idx_normal_signals[1]])
    ax[i].set_ylabel(labels[i])

In [ ]:
# Parameters to change
n_atoms = 3
atom_length = 500
penalty = 4

# Select a signal
signal = X_train[1, idx_normal_signals[0]]
signal = atleast_2d(signal)  # reshape

# Random number generator
rng = np.random.RandomState(seed=123)

In [ ]:
# get options for the optimizations
opt_dl = get_opt_dl(penalty=penalty)
opt_sc = get_opt_sc()

# Dictionary learning and sparse coding
dict_learning = cbpdndl.ConvBPDNDictLearn(
    D0=rng.randn(atom_length, 1, n_atoms),  # random init
    S=signal,  # signal at hand
    lmbda=penalty,  # sparsity penalty
    opt=opt_dl,  # options for the optimizations
    xmethod="admm",  # optimization method (sparse coding)
    dmethod="cns",  # optimization method (dict learnin)
)
atom_dictionary = dict_learning.solve()

# retrieve the sparse codes
basis_pursuit = cbpdn.ConvBPDN(
    D=atom_dictionary,  # learned dictionary
    S=signal,  # signal at hand
    lmbda=penalty,  # sparsity penalty
    opt=opt_sc,  # options for the optimizations
)
sparse_codes = basis_pursuit.solve().squeeze()

In [ ]:
plot_CDL(
    signal, atleast_2d(sparse_codes), atleast_2d(atom_dictionary.squeeze())
)

In [ ]:
# Reconstruction with the dictionary and the sparse codes
reconstruction = np.stack(
    [
        np.convolve(code, atom, mode="valid")
        for (code, atom) in zip(
            atleast_2d(sparse_codes).T, atleast_2d(atom_dictionary.squeeze()).T
        )
    ],
    axis=0,
)

# Note that the reconstruction has less samples than the original signal.
# This is because of border effects of the convolution.
offset = atom_length - 1

fig, ax = fig_ax()
tt = np.arange(signal.shape[0])
ax.plot(tt, signal, label="original", alpha=0.5)
ax.plot(tt[offset:], reconstruction.sum(axis=0), label="reconstructed")
ax.set_title(
    f"Reconstruction MSE: {np.mean((signal[offset :].flatten() - reconstruction.sum(axis=0))**2):.2e}"
)
_ = plt.legend()

Finding a dictionary is hard.
Let's use peaks

In [ ]:
def keep_peaks(peaks, treshold):
    time_diff_peaks = (peaks[1:] - peaks[:-1])/100.
    #plt.hist(time_diff_peaks,10)
    drop_values = np.where(time_diff_peaks < treshold)[0]
    time_diff_peaks = np.delete(time_diff_peaks, drop_values)
    return(np.delete(peaks,drop_values+1), time_diff_peaks)

In [ ]:
signal_id = idx_normal_signals[3383]

fig, ax = plt.subplots(nrows=4, figsize = (20,8))
for i in range(4):
    signal_i = X_train[i, signal_id]
    ax[i].plot(signal_i)
    treshold = signal_i.mean()+ signal_i.std()
    peaks, _ = sg.find_peaks(signal_i,height=treshold)
    peaks, time_peaks = keep_peaks(peaks, 0.6)
    print(f"Signal {i}")
    print(f"Average interval between peaks : {time_peaks.mean():.2f}, var : {time_peaks.var():.2f}, median : {np.median(time_peaks):.2f}, min : {time_peaks.min():.2f}, max : {time_peaks.max():.2f}  ")
    
    ax[i].plot(peaks, signal_i[peaks], "x")
    
    ax[i].plot(np.zeros_like(signal_i)+ treshold, "--", color="gray")

In [ ]:
signal_id = idx_normal_signals[1]

fig, ax = plt.subplots(nrows=4, figsize = (20,8))
for i in range(4):
    signal_i = X_train[i, signal_id]
    treshold = signal_i.mean()+ signal_i.std()
    ax[i].plot(signal_i)
    peaks, _ = sg.find_peaks(signal_i,height= treshold)
    peaks, time_peaks = keep_peaks(peaks, 0.6)
    print(f"Signal {i}")
    print(f"Average interval between peaks : {time_peaks.mean():.2f}, var : {time_peaks.var():.2f}, median : {np.median(time_peaks):.2f}, min : {time_peaks.min():.2f}, max : {time_peaks.max():.2f}  ")
    
    ax[i].plot(peaks, signal_i[peaks], "x")
    ax[i].plot(np.zeros_like(signal_i)+treshold, "--", color="gray")

In [ ]:
signal_id = idx_apnea_signals[37]
apneas_i = np.where(mask[signal_id] == 1)[0]

fig, ax = plt.subplots(nrows=4, figsize = (20,8))
for i in range(4):
    for idx in range(apneas_i.size):
        ax[i].axvline(apneas_i[idx]*100, color='#ffb6c1', linewidth=10)
    signal_i = X_train[i, signal_id]
    ax[i].plot(signal_i)
    treshold = signal_i.mean()+ signal_i.std()
    peaks, _ = sg.find_peaks(signal_i,height=treshold)
    peaks, time_peaks = keep_peaks(peaks, 0.6)
    print(f"Signal {i}")
    print(f"Average interval between peaks : {time_peaks.mean():.2f}, var : {time_peaks.var():.2f}, median : {np.median(time_peaks):.2f}, min : {time_peaks.min():.2f}, max : {time_peaks.max():.2f}  ")
    
    time_diff_peaks = []
    #peaks, _ = sg.find_peaks(signal_i,height=0)
    ax[i].plot(peaks, signal_i[peaks], "x")
    ax[i].plot(np.zeros_like(signal_i)+treshold, "--", color="gray")
    ax[i].set_ylabel(labels[i])
    
    

In [ ]:
signal_id = idx_apnea_signals[100]
apneas_i = np.where(mask[signal_id] == 1)[0]

fig, ax = plt.subplots(nrows=4, figsize = (20,8))
for i in range(4):
    for idx in range(apneas_i.size):
        ax[i].axvline(apneas_i[idx]*100, color='#ffb6c1', linewidth=10)
    signal_i = X_train[i, signal_id]
    ax[i].plot(signal_i)
    treshold = signal_i.mean()+ signal_i.std()
    peaks, _ = sg.find_peaks(signal_i,height=treshold)
    peaks, mean = keep_peaks(peaks, 0.6)
    print(mean)
    
    time_diff_peaks = []
    #peaks, _ = sg.find_peaks(signal_i,height=0)
    ax[i].plot(peaks, signal_i[peaks], "x")
    ax[i].plot(np.zeros_like(signal_i)+treshold, "--", color="gray")
    ax[i].set_ylabel(labels[i])
    
    

In [ ]:
signal_id = idx_apnea_signals[500]
apneas_i = np.where(mask[signal_id] == 1)[0]

fig, ax = plt.subplots(nrows=4, figsize = (20,8))
for i in range(4):
    for idx in range(apneas_i.size):
        ax[i].axvline(apneas_i[idx]*100, color='#ffb6c1', linewidth=10)
    signal_i = X_train[i, signal_id]
    ax[i].plot(signal_i)
    treshold = signal_i.mean()+ signal_i.std()
    peaks, _ = sg.find_peaks(signal_i,height=treshold)
    peaks, time_peaks = keep_peaks(peaks, 0.6)
    print(f"Signal {i}")
    print(f"Average interval between peaks : {time_peaks.mean():.2f}, var : {time_peaks.var():.2f}, median : {np.median(time_peaks):.2f}, min : {time_peaks.min():.2f}, max : {time_peaks.max():.2f}  ")
    
    time_diff_peaks = []
    #peaks, _ = sg.find_peaks(signal_i,height=0)
    ax[i].plot(peaks, signal_i[peaks], "x")
    ax[i].plot(np.zeros_like(signal_i)+treshold, "--", color="gray")
    ax[i].set_ylabel(labels[i])
    
    

In [ ]:
def peak_features_extractor(signal_id, dim, n_window=500, peak_param=1, time_treshold=0.5, plot=False):
    
    '''
    Returns matrix of shape 9000 which computes the distance to the closest peak, as well as the
    average value of the signal on a window of size n_window
    
    '''
    
    signal_i = X_train[dim, signal_id]
    treshold = signal_i.mean()+ peak_param*signal_i.std()
    peaks, _ = sg.find_peaks(signal_i,height=treshold)
    peaks, mean = keep_peaks(peaks, time_treshold)
    peaks_added = np.zeros(2 + peaks.size)
    peaks_added[1:-1] = peaks
    peaks_added[-1] = signal_i.size-1
    
    dist_matrix = np.zeros((signal_i.size,3))
    mean_window = np.ones(n_window) / n_window
    
    next_peak_idx = 1
    prev_peak_idx = 0
    
    
    for i in range(signal_i.size):
        dist_matrix[i,0] = (i - peaks_added[prev_peak_idx])/100
        dist_matrix[i,1] = (peaks_added[next_peak_idx] - i)/100
        
        if i == peaks_added[next_peak_idx]:
            next_peak_idx +=1
            prev_peak_idx +=1
            
    
    dist_matrix[:, 2] = np.convolve(signal_i, n_window, 'same' )
    
    
    if plot:
        fig, ax = plt.subplots(figsize=(20,5))
        apneas_i = np.where(mask[signal_id] == 1)[0]
        for idx in range(apneas_i.size):
            ax.axvline(apneas_i[idx]*100, color='#ffb6c1', linewidth=10)
        
        ax.plot(signal_i)
        ax.plot(peaks, signal_i[peaks], "x")
        ax.plot(np.zeros_like(signal_i)+treshold, "--", color="gray")
        ax.set_ylabel(labels[dim])
        
    
    return(dist_matrix)
        

bandpass filters on eeg

In [6]:

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
signal_id = idx_apnea_signals[500]
apneas_i = np.where(mask[signal_id] == 1)[0]

fig, ax = plt.subplots(nrows=4, figsize = (20,8))
for i in range(4):
    for idx in range(apneas_i.size):
        ax[i].axvline(apneas_i[idx]*100, color='#ffb6c1', linewidth=10)
    signal_i = X_train[i, signal_id]
    ax[i].plot(signal_i)
    treshold = signal_i.mean()+ signal_i.std()
    peaks, _ = sg.find_peaks(signal_i,height=treshold)
    peaks, time_peaks = keep_peaks(peaks, 0.6)
    print(f"Signal {i}")
    print(f"Average interval between peaks : {time_peaks.mean():.2f}, var : {time_peaks.var():.2f}, median : {np.median(time_peaks):.2f}, min : {time_peaks.min():.2f}, max : {time_peaks.max():.2f}  ")
    
    time_diff_peaks = []
    #peaks, _ = sg.find_peaks(signal_i,height=0)
    ax[i].plot(peaks, signal_i[peaks], "x")
    ax[i].plot(np.zeros_like(signal_i)+treshold, "--", color="gray")
    ax[i].set_ylabel(labels[i])
    
    

In [ ]:
y_alpha = butter_bandpass_filter(signal_apnea, 8, 12, fs, order=3)
y_beta = butter_bandpass_filter(signal_apnea, 16, 40, fs, order=3)
y_sigma = butter_bandpass_filter(signal_apnea, 12, 16, fs, order=3)
y_delta = butter_bandpass_filter(signal_apnea, 0.25, 4, fs, order=3)
y_theta = butter_bandpass_filter(signal_apnea, 4, 8, fs, order=3)

y_bands = np.asarray([y_delta, y_theta, y_alpha, y_sigma, y_beta])


In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
#ax.plot(y_delta, label="delta: 0.25-4Hz")
ax.plot(y_theta, label="theta: 4-8Hz")
ax.plot(y_alpha, label="alpha: 8-12Hz ")
ax.plot(y_sigma, label="sigma: 12-16Hz")
ax.plot(y_beta, label="beta: 16-40Hz")
ax.plot(signal_apnea, c='k', label='signal')
ax.legend()

In [ ]:
#plt.plot(y_alpha)
#plt.plot(y_beta)
#plt.plot(y_delta)

fig, (ax,ax1) = plt.subplots(nrows = 2, figsize=(20,10))
ax.plot(y_delta[:300], label="delta: 0.25-4Hz")
ax.plot(y_theta[:300], label="theta: 4-8Hz")
ax.plot(y_alpha[:300], label="alpha: 8-12Hz ")
ax.plot(y_sigma[:300], label="sigma: 12-16Hz")
ax.plot(y_beta[:300], label="beta: 16-40Hz")
ax.plot(signal_apnea[:300], c='k', label='signal')
ax.legend()
ax.set_title("Zoom sur le signal pendant le sommeil normal")

#ax1.plot(np.cumsum(y_delta[:100]**2), label="delta: 0.25-4Hz")
ax1.plot(y_delta[2500:2800], label="delta: 0.25-4Hz")
ax1.plot(y_theta[2500:2800], label="theta: 4-8Hz")
ax1.plot(y_alpha[2500:2800], label="alpha: 8-12Hz ")
ax1.plot(y_sigma[2500:2800], label="sigma: 12-16Hz")
ax1.plot(y_beta[2500:2800], label="beta: 16-40Hz")
ax1.plot(signal_apnea[2500:2800], c='k', label='signal')
ax1.legend()
ax1.set_title("Zoom sur le signal pendant une épisode d'apnée")

In [7]:
from scipy.stats import mode, skew

signals = ["d", "t", "a", "s", "b"]
features = ["energy", "entropy", "variance", "std", "mean", "median", "mode", "min", "max"]

def energy(signal):
    return(np.sum(signal**2))

def entropy(signal):
    return(np.sum(signal**2*np.log(signal**2)))

def features_subsignal(signal):
    
    ener = energy(signal)
    entro = entropy(signal)
    var = signal.var()
    std = signal.std()
    mean = signal.mean()
    median = np.median(signal)
    mod = mode(signal)[0][0]
    mini = signal.min()
    maxi = signal.max()
    
    return(np.asarray([ener,entro,var,std,mean,median,mod,mini,maxi]))

def features_signal(signal, half_window_size=150, window_overlap=15):
    
    l_features = []
    
    i=0
    cnt =0
    while i < half_window_size:
        l_features += [features_subsignal(signal[:i + half_window_size])]
        i += window_overlap
        cnt +=1

    while i + half_window_size < signal.size :
        l_features += [features_subsignal(signal[i-half_window_size: i + half_window_size ])]
        i += window_overlap
        cnt +=1
    while i < signal.size:
        l_features += [features_subsignal(signal[i-half_window_size: ])]
        i += window_overlap
        cnt +=1
    return(np.asarray(l_features))
        
        


In [8]:
def peak_features_extraction(signal_id, half_window_size=150, window_overlap= 15, n_window=100, peak_param=1, time_treshold=0.5):
    
    n_samples, idx = compute_idx(half_window_size, window_overlap,9000)
    
    feature_matrix = np.zeros((n_samples, 3*4))
    
    for i in range(4):
        feature_matrix[:, 3*i:3*(i+1)] = peak_features_extractor(signal_id, dim=i, n_window=n_window,
                                                peak_param=peak_param, time_treshold=time_treshold)[idx]
        
    return(feature_matrix)

In [ ]:
eeg_bands = np.zeros((4400,9000,12))


for i in range(4400):
    eeg_bands[i] = signal_features_extractor(i)
    if i%100 == 0:
        print(f"Extracted features of {i+1} signals")
    

In [10]:
def peak_features_extractor(signal_id, dim, n_window=100, peak_param=1, time_treshold=0.5, plot=False):
    
    '''
    Returns matrix of shape 9000 which computes the distance to the closest peak, as well as the
    average value of the signal on a window of size n_window
    
    '''
    
    signal_i = X_train[dim, signal_id]
    treshold = signal_i.mean()+ peak_param*signal_i.std()
    peaks, _ = sg.find_peaks(signal_i,height=treshold)
    peaks, mean = keep_peaks(peaks, time_treshold)
    peaks_added = np.zeros(2 + peaks.size)
    peaks_added[1:-1] = peaks
    peaks_added[-1] = signal_i.size-1
    
    dist_matrix = np.zeros((signal_i.size,3))
    mean_window = np.ones(n_window) / n_window
    
    next_peak_idx = 1
    prev_peak_idx = 0
    
    
    for i in range(signal_i.size):
        dist_matrix[i,0] = (i - peaks_added[prev_peak_idx])/100
        dist_matrix[i,1] = (peaks_added[next_peak_idx] - i)/100
        
        if i == peaks_added[next_peak_idx]:
            next_peak_idx +=1
            prev_peak_idx +=1
            
    
    dist_matrix[:, 2] = np.convolve(signal_i, n_window, 'same' )
    
    
    if plot:
        fig, ax = plt.subplots(figsize=(20,5))
        apneas_i = np.where(mask[signal_id] == 1)[0]
        for idx in range(apneas_i.size):
            ax.axvline(apneas_i[idx]*100, color='#ffb6c1', linewidth=10)
        
        ax.plot(signal_i)
        ax.plot(peaks, signal_i[peaks], "x")
        ax.plot(np.zeros_like(signal_i)+treshold, "--", color="gray")
        ax.set_ylabel(labels[dim])
        
    
    return(dist_matrix)
        

In [11]:
def compute_idx(half_window, overlap, length):
    cnt = 0
    idx = []
    i=0
    while i + half_window < length:
        cnt+=1
        idx +=[i]
        i += overlap
    while i < length:
        cnt+=1
        idx +=[i]
        i += overlap
        
    return(cnt, idx)
        

## BAND EXTRACTION

In [ ]:
def signal_features_extractor(signal_idx, fs=100):
    """
    Order is
    - First EEG sinal
    - delta
    - theta
    - alpha
    - sigma
    - beta
    
    - Second EEG signal
    (same order for the bands)
    
    """
    
    
    feature_matrix = np.zeros((9000, 6*2))
    feature_matrix[:,0] = X_train[6, signal_idx]
    feature_matrix[:,1] = butter_bandpass_filter(X_train[6, signal_idx], 0.25, 4, fs, order=3)
    feature_matrix[:,2] = butter_bandpass_filter(X_train[6, signal_idx], 4, 8, fs, order=3)
    feature_matrix[:,3] = butter_bandpass_filter(X_train[6, signal_idx], 8, 12, fs, order=3)
    feature_matrix[:,4] = butter_bandpass_filter(X_train[6, signal_idx], 16, 40, fs, order=3)
    feature_matrix[:,5] = butter_bandpass_filter(X_train[6, signal_idx], 12, 16, fs, order=3)
    

    feature_matrix[:,6] = X_train[7, signal_idx]
    feature_matrix[:,7] = butter_bandpass_filter(X_train[7, signal_idx], 0.25, 4, fs, order=3)
    feature_matrix[:,8] = butter_bandpass_filter(X_train[7, signal_idx], 4, 8, fs, order=3)
    feature_matrix[:,9] = butter_bandpass_filter(X_train[7, signal_idx], 8, 12, fs, order=3)
    feature_matrix[:,10] = butter_bandpass_filter(X_train[7, signal_idx], 16, 40, fs, order=3)
    feature_matrix[:,11] = butter_bandpass_filter(X_train[7, signal_idx], 12, 16, fs, order=3)
        
        #feature_matrix[:, col_idx:col_idx+9] = features_signal(y_alpha, half_window_size, window_overlap)
        #col_idx +=9
        #feature_matrix[:, col_idx:col_idx+9] = features_signal(y_beta, half_window_size, window_overlap)
        #col_idx +=9
        #feature_matrix[:, col_idx:col_idx+9] = features_signal(y_sigma, half_window_size, window_overlap)
        #col_idx +=9
        #feature_matrix[:, col_idx:col_idx+9] = features_signal(y_delta, half_window_size, window_overlap)
        #col_idx +=9
        #feature_matrix[:, col_idx:col_idx+9] = features_signal(y_theta, half_window_size, window_overlap)
        #col_idx +=9
    return(feature_matrix)
    
    

In [12]:
eeg_bands = np.zeros((4400,9000,12))


for i in range(4400):
    eeg_bands[i] = signal_features_extractor(i)
    if i%100 == 0:
        print(f"Extracted features of {i+1} signals")
    

Extracted features of 1 signals
Extracted features of 101 signals
Extracted features of 201 signals
Extracted features of 301 signals
Extracted features of 401 signals
Extracted features of 501 signals
Extracted features of 601 signals
Extracted features of 701 signals
Extracted features of 801 signals
Extracted features of 901 signals
Extracted features of 1001 signals
Extracted features of 1101 signals
Extracted features of 1201 signals
Extracted features of 1301 signals
Extracted features of 1401 signals
Extracted features of 1501 signals
Extracted features of 1601 signals
Extracted features of 1701 signals
Extracted features of 1801 signals
Extracted features of 1901 signals
Extracted features of 2001 signals
Extracted features of 2101 signals
Extracted features of 2201 signals
Extracted features of 2301 signals
Extracted features of 2401 signals
Extracted features of 2501 signals
Extracted features of 2601 signals
Extracted features of 2701 signals
Extracted features of 2801 signa

In [13]:
h5f = h5py.File('features/bands_train_patients.h5', 'w')
h5f.create_dataset('data', data=eeg_bands)
h5f.close()